In [9]:
%load_ext autoreload
%autoreload 2
from load import load_3D
from liftpose.vision_3d import XY_coord_dict, Z_coord_dict
import yaml
import logging
from imp import reload
import torch
import numpy as np
import matplotlib.pyplot as plt
import os 
import glob
import torch
import pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib
from matplotlib.animation import FFMpegWriter
from matplotlib.legend_handler import HandlerTuple
matplotlib.use('Agg')
from skeleton import skeleton
from tqdm import tqdm
import plotting as plotting
import utils as utils
import stats as stat
import yaml
import sys

logger = logging.getLogger(__name__).setLevel(logging.INFO)

# decleare data parameters
par_train = {'data_dir'       : './out', # change the path 
             'out_dir'        : './out',
             "train_subjects" : ["004"],
             'test_subjects'  : ["002"],
             'actions'        : ['PR']}

# merge with training parameters
par_data = yaml.full_load(open('param.yaml', "rb"))
par = {**par_data["data"], **par_train}

#load
G, color_edge = skeleton()
legtips = [4, 9, 14, 19, 24, 29]
data = torch.load(par['data_dir'] + '/test_results.pth.tar')

tar_mean = torch.load(par['data_dir'] + '/stat_3d.pth.tar')['mean']
tar_std = torch.load(par['data_dir'] + '/stat_3d.pth.tar')['std']
targets_1d = torch.load(par['data_dir'] + '/stat_3d.pth.tar')['targets_3d']
tar_offset = np.vstack(torch.load(par['data_dir'] + '/stat_3d.pth.tar')['offset'].values())

inp_mean = torch.load(par['data_dir'] + '/stat_2d.pth.tar')['mean']
inp_std = torch.load(par['data_dir'] + '/stat_2d.pth.tar')['std']
targets_2d = torch.load(par['data_dir'] + '/stat_2d.pth.tar')['targets_2d']
inp_offset = np.vstack(torch.load(par['data_dir'] + '/stat_2d.pth.tar')['offset'].values())[0,:]

good_keypts = utils.add_roots(data['good_keypts'],targets_1d,len(tar_mean))
if np.sum(good_keypts[0,:15])>10:
    tar_offset = np.hstack((tar_offset[0,:15],tar_offset[0,:15]))
else:
    tar_offset = np.hstack((tar_offset[0,15:],tar_offset[0,15:]))

#unnormalize
tar = stat.unNormalize(data['target'], tar_mean[targets_1d], tar_std[targets_1d])
tar = utils.add_roots(tar,targets_1d,len(tar_mean))
tar += tar_offset
out = stat.unNormalize(data['output'], tar_mean[targets_1d], tar_std[targets_1d])
out = utils.add_roots(out,targets_1d,len(tar_mean))
out += tar_offset
inp = stat.unNormalize(data['input'], inp_mean[targets_2d], inp_std[targets_2d])
inp = utils.add_roots(inp,targets_2d,len(inp_mean))
inp += inp_offset

inp = utils.filter_data(inp, window=5, order=2)
out = utils.filter_data(out, window=5, order=2)
tar = utils.filter_data(tar, window=5, order=2)

# Set up a figure
fig = plt.figure(figsize=plt.figaspect(1))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=40, azim=140)

writer = FFMpegWriter(fps=10)
xlim, ylim, zlim = None,None,None
with writer.saving(fig, "LiftPose3D_prediction_fly_prism.mp4", 100):
    for t in tqdm(range(800,950)):
        
        plt.cla()
        
        thist = 5
        pos_pred, pos_tar = [], []
        for j in range(out.shape[1]):
            tmin = max(0,t-thist+1)
            pos_pred.append((inp[tmin:(t+1), 2*j], inp[tmin:(t+1), 2*j+1], out[tmin:(t+1), j]))
            pos_tar.append((inp[tmin:(t+1), 2*j], inp[tmin:(t+1), 2*j+1], tar[tmin:(t+1), j]))
                
        pos_pred, pos_tar = np.array(pos_pred), np.array(pos_tar)
                    
        #plot trailing dots
        plotting.plot_trailing_points(pos_pred[legtips,:,:],min(thist,t+1),ax)
        
        #plot skeleton
        plotting.plot_3d_graph(G, pos_tar[:,:,-1], ax, color_edge=color_edge, good_keypts=good_keypts[t,:])    
        plotting.plot_3d_graph(G, pos_pred[:,:,-1], ax, color_edge=color_edge, style='--') 

        if xlim is None:
            xlim = ax.get_xlim()
            ylim = ax.get_ylim()
            zlim = ax.get_zlim()
        
        #### this bit is just to make special legend 
        pts = np.array([1,1])
        p1, = ax.plot(pts, pts, pts, 'r-')
        p2, = ax.plot(pts, pts, pts, 'b-')
        p3, = ax.plot(pts, pts, pts, 'r--', dashes=(2, 2))
        p4, = ax.plot(pts, pts, pts, 'b--', dashes=(2, 2))
        ax.legend([(p1, p2), (p3, p4)], 
            ['Triangulated 3D pose using both the ventral/side views', \
             'LiftPose3D prediction using only the ventral view'], 
            numpoints=1, handler_map={tuple: HandlerTuple(ndivide=None)},
            loc=(0.05,0.9),
            frameon=False)    
        p1.remove()
        p2.remove()
        p3.remove()
        p4.remove()
        ####
        
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_zlim(zlim)
        
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])
        ax.grid(True)
    
        writer.grab_frame()

/home/user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/home/user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:55: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  0%|          | 0/150 [00:00<?, ?it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 150/150 [00:16<00:00,  9.27it/s]


In [14]:
!ffmpeg -i DLC_prediction_fly_prism.mp4 -i LiftPose3D_prediction_fly_prism.mp4 -filter_complex "[0:v]scale=-1:400[v0];[v0][1:v]hstack=inputs=2" Video_4.mp4 -y

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavforma